In [1]:
import boto3

In [2]:
# constants
BUCKET_NAME = 'datacont-name-234'
AWS_ACCESS_KEY_ID = 'SOME_ID'
AWS_SECRET_ACCESS_KEY = 'SOME_KEY'

In [3]:
s3 = boto3.resource('s3',
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY)

In [4]:
# create bucket
try:
    s3.create_bucket(Bucket=BUCKET_NAME, CreateBucketConfiguration={
    'LocationConstraint': 'us-west-2'}) 
except Exception as e:
    print (e)

An error occurred (BucketAlreadyOwnedByYou) when calling the CreateBucket operation: Your previous request to create the named bucket succeeded and you already own it.


In [5]:
# get bucket
bucket = s3.Bucket(BUCKET_NAME)

In [6]:
# make bucket publicly readable
bucket.Acl().put(ACL='public-read')

{'ResponseMetadata': {'RequestId': 'HJNY29184SP9S4T1',
  'HostId': 'eALBpjcYt6jHT3xfuhVwciI3N4B/pk5oQ+XyqZXifC9VL96xci9tEqVfFmMnU6vF5DJQelmLSso=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'eALBpjcYt6jHT3xfuhVwciI3N4B/pk5oQ+XyqZXifC9VL96xci9tEqVfFmMnU6vF5DJQelmLSso=',
   'x-amz-request-id': 'HJNY29184SP9S4T1',
   'date': 'Wed, 20 Oct 2021 21:08:49 GMT',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0}}

In [7]:
# upload exp1.csv into the bucket
exp1 = open('exp1.csv', 'rb')
o = s3.Object(BUCKET_NAME, 'exp1.csv').put(Body=exp1)
s3.Object(BUCKET_NAME, 'exp1.csv').Acl().put(ACL='public-read')

{'ResponseMetadata': {'RequestId': 'S454DNZSA3MDC3Y4',
  'HostId': 'o5co3TlgHQY9c+2cyCNgq2a7rfaYRZ2H+uLvJfAUAmJb0O4uHM2W8YIs0qzrtTiXYChCrZCIBf8=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'o5co3TlgHQY9c+2cyCNgq2a7rfaYRZ2H+uLvJfAUAmJb0O4uHM2W8YIs0qzrtTiXYChCrZCIBf8=',
   'x-amz-request-id': 'S454DNZSA3MDC3Y4',
   'date': 'Wed, 20 Oct 2021 21:10:59 GMT',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0}}

In [8]:
# upload exp2.csv into the bucket
exp2 = open('exp2.csv', 'rb')
o = s3.Object(BUCKET_NAME, 'exp2.csv').put(Body=exp2)
s3.Object(BUCKET_NAME, 'exp2.csv').Acl().put(ACL='public-read')

{'ResponseMetadata': {'RequestId': 'HK32Z98ER4QFZZQ8',
  'HostId': 'QpnFzks8PJ2G3KaLsP0QHwuyfwgJU3fFChF/HRWhnqImdiI3yxygnlwiujgW/hQaBg0LEq/n7iI=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'QpnFzks8PJ2G3KaLsP0QHwuyfwgJU3fFChF/HRWhnqImdiI3yxygnlwiujgW/hQaBg0LEq/n7iI=',
   'x-amz-request-id': 'HK32Z98ER4QFZZQ8',
   'date': 'Wed, 20 Oct 2021 21:12:02 GMT',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0}}

In [9]:
# upload exp3.csv into the bucket
exp3 = open('exp3.csv', 'rb')
o = s3.Object(BUCKET_NAME, 'exp3.csv').put(Body=exp3)
s3.Object(BUCKET_NAME, 'exp3.csv').Acl().put(ACL='public-read')

{'ResponseMetadata': {'RequestId': 'J1GXFARV3YE7CQHW',
  'HostId': 'EMloBiZ7P/VcHL0IB+UXGzA23rbSNs/Mmux0a+b9TPvR/e+Khh9VdY3cP6mtvm6KkmEpgFMjyus=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'EMloBiZ7P/VcHL0IB+UXGzA23rbSNs/Mmux0a+b9TPvR/e+Khh9VdY3cP6mtvm6KkmEpgFMjyus=',
   'x-amz-request-id': 'J1GXFARV3YE7CQHW',
   'date': 'Wed, 20 Oct 2021 21:12:03 GMT',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0}}

In [10]:
# create dynamodb table
dyndb = boto3.resource('dynamodb',
    region_name='us-west-2',
    aws_access_key_id=AWS_ACCESS_KEY_ID, 
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY
)

In [11]:
# create table
try:
    table = dyndb.create_table(
        TableName='DataTable3',
        KeySchema=[
            {
                'AttributeName': 'PartitionKey',
                'KeyType': 'HASH'
            }
            , 
            {
                'AttributeName': 'RowKey',
                'KeyType': 'RANGE'
            }
        ],
        AttributeDefinitions=[
            {
                'AttributeName': 'PartitionKey',
                'AttributeType': 'S'
            }
            , 
            {
                'AttributeName': 'RowKey',
                'AttributeType': 'S'
            },
        ],
        ProvisionedThroughput={
            'ReadCapacityUnits': 5,
            'WriteCapacityUnits': 5
        }
    )
except Exception as e:
    print (e)
    #if there is an exception, the table may already exist.
    table = dyndb.Table("DataTable")


In [12]:
# wait for the table to be created
table.meta.client.get_waiter('table_exists').wait(TableName='DataTable3')

In [13]:
print(table.item_count)

0


In [14]:
import csv

In [15]:
with open('experiments.csv', 'rt') as csvfile: 
    csvf = csv.reader(csvfile, delimiter=',', quotechar='|')
    next(csvf)
    for item in csvf:
        print(item)
        body = open(item[4], 'rb') 
        s3.Object(BUCKET_NAME, item[4]).put(Body=body) 
        md = s3.Object(BUCKET_NAME, item[4]).Acl().put(ACL='public-read')
        
        url = " https://s3-us-west-2.amazonaws.com/"+BUCKET_NAME+"/"+item[4] 
        metadata_item = {'PartitionKey': item[0], 'RowKey': item[1],
            'Conductivity' : item[2], 'Concentration' : item[3], 'url':url} # 'RowKey': item[1]
        try: 
            table.put_item(Item=metadata_item)
        except:
            print("item may already be there or another failure")

['1', '-1', '52', '3.4', 'exp1.csv']
['2', '-2', '52.1', '3.4', 'exp2.csv']
['3', '-2.93', '57.1', '3.7', 'exp3.csv']


In [17]:
response = table.get_item(
    Key={
        'PartitionKey': '1',
        'RowKey': '-1'
    }
)
item = response['Item'] 
print(item)


{'Concentration': '3.4', 'PartitionKey': '1', 'RowKey': '-1', 'url': ' https://s3-us-west-2.amazonaws.com/datacont-name-234/exp1.csv', 'Conductivity': '52'}


In [19]:
print(response)

{'Item': {'Concentration': '3.4', 'PartitionKey': '1', 'RowKey': '-1', 'url': ' https://s3-us-west-2.amazonaws.com/datacont-name-234/exp1.csv', 'Conductivity': '52'}, 'ResponseMetadata': {'RequestId': 'NSMAHAOCGLC4AF7ICHLR6UAR0JVV4KQNSO5AEMVJF66Q9ASUAAJG', 'HTTPStatusCode': 200, 'HTTPHeaders': {'server': 'Server', 'date': 'Wed, 20 Oct 2021 21:34:38 GMT', 'content-type': 'application/x-amz-json-1.0', 'content-length': '186', 'connection': 'keep-alive', 'x-amzn-requestid': 'NSMAHAOCGLC4AF7ICHLR6UAR0JVV4KQNSO5AEMVJF66Q9ASUAAJG', 'x-amz-crc32': '3815956703'}, 'RetryAttempts': 0}}
